In [1]:
import numpy as np
from catboost import CatBoostClassifier

In [2]:
X_train = np.load('X_train.npy')
y = np.load('y.npy')

In [ ]:
class GiniMetric(object):
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)

    def is_max_optimal(self):
        return True

    def evaluate(self, predictions, labels, weight):
        preds = predictions[0]
        assert len(predictions) == 1
        assert len(labels) == len(preds)

        error_sum = 0.0
        weight_sum = 0.0

        for idx, (a, p) in enumerate(zip(labels, preds)):
            w = 1.0 if weight is None else weight[idx]
            weight_sum += w
            error_sum += w * (ngini(a, p))

        return error_sum, weight_sum

In [5]:
model = CatBoostClassifier(iterations=5, depth=2, learning_rate=1, loss_function=GiniMetric(), verbose=True)

In [6]:
model.fit(X_train,y)

Borders for float features generated
0:	learn 0.1564648792	total: 1.24s	remaining: 4.95s
1:	learn 0.1549364458	total: 2.38s	remaining: 3.57s
2:	learn 0.154234155	total: 3.56s	remaining: 2.37s
3:	learn 0.1537520105	total: 4.69s	remaining: 1.17s
4:	learn 0.1536428475	total: 5.86s	remaining: 0us


In [ ]:
X_test = np.load('X_test.npy')
preds_proba = model.predict_proba(X_test)

In [10]:
preds_proba.shape

(892816, 2)

In [11]:
test

(892816, 58)

In [17]:
import pandas as pd
#test = pd.read_csv('test.csv')
test['target'] = preds_proba[:,1]
test[['id','target']].to_csv('submission.csv', index=False, float_format='%.5f')